ELMo의 가장 큰 특징은 **사전 훈련된 언어 모델(Pre-trained language model)**을 사용한다는 점이다.
*하지만 현재 텐서플로우 2.0에서는 사용 불가, 텐서플로우 버전 1버전에서 사용해야*

## 1. ELMo(Embeddings from Language Model)
같은 표기의 단어인데 문맥에 따라 전혀 다른 의미로 사용되는 것은 Word2Vec와 GloVe가 제대로 반영하지 못한다.

**문맥을 반영한 워드 임베딩(Contextualized Word Embedding)** 등장


## 2. biLM(Bidirectional Language Model)의 사전 훈련
* biLM은 기본적으로 다층 구조(Multi-layer)를 전제로 한다.
* char CNN이라는 방법을 사용해 글자(character) 단위로 계산, 서브단어(subword)의 정보를 참고하는 것처럼 문맥과 상관없이 단어 간의 연관성을 찾아낸다.
* OOV에도 견고하다.

*양방향 RNN과 ELMo에서의 biLM은 다르다. 양방향 RNN은 순방향 RNN의 은닉 상태와 역방향의 RNN의 은닉 상태를 다음 층의 입력으로 보내기 전에 연결(concatenate). biLM의 순방향 언어모델과 역방향 언어모델이 각각의 은닉 상태만을 다음 은닉층으로 보내며 훈련시킨 후에 ELMo 표현으로 사용하기 위해서 은닉 상태를 연결(concatenate).*


## 3. biLM의 활용
biLM이 훈련되었다면, ELMo가 사전 훈련된 biLM을 통해 입력 문장으로부터 단어를 임베딩하기 위한 과정을 살펴보자.

1. 해당 시점(time-step)의 BiLM의 각 층의 출력값을 가져온다.
2. 순방향 언어 모델과 역방향 언어 모델의 **각 층의 출력값을 연결(concatenate)**한다.
  * 각 층의 출력값이란, 첫번째는 임베딩 층, 나머지 층은 각 층의 은닉 상태를 의미
  * ELMo의 아이디어로, *각 층의 출력값이 가진 정보는 전부 다른 정보일 것이므로 이들을 모두 활용한다는 점이다.*
3. **각 층의 출력값 별로 가중치를 준다.**
4. **각 층의 출력값을 모두 더한다.** (3,4번은 가중합(Weighted Sum)이다.)
5. **벡터의 크기를 결정하는 스칼라 매개변수를 곱한다.**

=> ELMo 표현(representation) 완성

* ELMo 표현은 기존의 임베딩 벡터와 함께 사용할 수 있다.

## 4. ELMo 표현을 사용해서 스팸 메일 분류하기

*텐서플로우 1버전 설정*

In [1]:
%tensorflow_version 1.x 

TensorFlow 1.x selected.


In [2]:
# 텐서플로우 허브로부터 사전 훈련된 모델 사용
!pip install tensorflow-hub

In [3]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K
import urllib.request
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [4]:
# 텐서플로우 허브로부터 ELMo를 다운로드
elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)

sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [5]:
# 스팸 메일 분류하기 데이터를 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin-1')
data[:5]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
# 데이터 전처리
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
y_data = list(data['v1'])
X_data = list(data['v2'])

In [7]:
X_data[:5]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]

In [8]:
y_data[:5]

[0, 0, 1, 0, 0]

In [9]:
# 훈련 데이터와 테스트 데이터 80%, 20%로 분리
print(len(X_data))
n_of_train = int(len(X_data) * 0.8)
n_of_test = int(len(X_data) - n_of_train)
print(n_of_train)
print(n_of_test)

5572
4457
1115


In [10]:
X_train = np.asarray(X_data[:n_of_train]) 
y_train = np.asarray(y_data[:n_of_train])
X_test = np.asarray(X_data[n_of_train:])
y_test = np.asarray(y_data[n_of_train:]) 

In [11]:
# ELMo는 텐서플로우 허브로부터 가져온 것이기 때문에 케라스에서 사용하기 위해서는 케라스에서 사용할 수 있도록 변환해주는 작업들이 필요
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict=True, signature="default")["default"]
# 데이터의 이동이 케라스 → 텐서플로우 → 케라스가 되도록 하는 함수

In [14]:
# 모델 설계
from keras.models import Model
from keras.layers import Dense, Lambda, Input

input_text = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(hidden_layer)
model = Model(inputs=[input_text], outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [15]:
history = model.fit(X_train, y_train, epochs=1, batch_size=60)

Epoch 1/1
4457/4457 [==============================] - 48s 11ms/step - loss: 0.1315 - accuracy: 0.9531


In [16]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

1115/1115 [==============================] - 11s 10ms/step

 테스트 정확도: 0.9749
